In [24]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
from sklearn.impute import KNNImputer,SimpleImputer
kn  = KNNImputer(n_neighbors=10)
%matplotlib inline
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import log_loss
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier


print('done')

ModuleNotFoundError: No module named 'optuna'

In [23]:
pip install catboost

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
train.nunique()

Artist Name            9149
Track Name            15129
Popularity              100
danceability            909
energy                 1208
key                      11
loudness               9002
mode                      2
speechiness            1194
acousticness           3930
instrumentalness       4253
liveness               1459
valence                1320
tempo                 13562
duration_in min/ms    14121
time_signature            4
Class                    11
dtype: int64

In [4]:
train.isnull().sum()

Artist Name              0
Track Name               0
Popularity             428
danceability             0
energy                   0
key                   2014
loudness                 0
mode                     0
speechiness              0
acousticness             0
instrumentalness      4377
liveness                 0
valence                  0
tempo                    0
duration_in min/ms       0
time_signature           0
Class                    0
dtype: int64

In [5]:
# Preprocessing and feature extraction
dur = train['duration_in min/ms']
track = []
clean = []
for i in dur:
    if i<=100 :
        clean.append(i)
        track.append(0)
    else:
        clean.append(i/60000)
        track.append(1)
train['duration_in min/ms'] = clean
train['new1'] = track

In [6]:
train['new1'].sample(5)

14898    1
11942    1
2402     0
1411     1
15523    1
Name: new1, dtype: int64

In [7]:
check_missing = train.isnull()
for column in check_missing.columns.values.tolist():
    print(column)
    print(check_missing[column].value_counts())

Artist Name
False    17996
Name: Artist Name, dtype: int64
Track Name
False    17996
Name: Track Name, dtype: int64
Popularity
False    17568
True       428
Name: Popularity, dtype: int64
danceability
False    17996
Name: danceability, dtype: int64
energy
False    17996
Name: energy, dtype: int64
key
False    15982
True      2014
Name: key, dtype: int64
loudness
False    17996
Name: loudness, dtype: int64
mode
False    17996
Name: mode, dtype: int64
speechiness
False    17996
Name: speechiness, dtype: int64
acousticness
False    17996
Name: acousticness, dtype: int64
instrumentalness
False    13619
True      4377
Name: instrumentalness, dtype: int64
liveness
False    17996
Name: liveness, dtype: int64
valence
False    17996
Name: valence, dtype: int64
tempo
False    17996
Name: tempo, dtype: int64
duration_in min/ms
False    17996
Name: duration_in min/ms, dtype: int64
time_signature
False    17996
Name: time_signature, dtype: int64
Class
False    17996
Name: Class, dtype: int64
new1
F

In [8]:
test.isnull().sum()

Artist Name              0
Track Name               0
Popularity             227
danceability             0
energy                   0
key                    808
loudness                 0
mode                     0
speechiness              0
acousticness             0
instrumentalness      1909
liveness                 0
valence                  0
tempo                    0
duration_in min/ms       0
time_signature           0
dtype: int64

In [9]:
train['key']=kn.fit_transform(train[['key']])
train['Popularity']=kn.fit_transform(train[['Popularity']])
train.replace(to_replace = np.nan, value =0,inplace=True)

In [10]:
test['key']=kn.fit_transform(test[['key']])
test['Popularity']=kn.fit_transform(test[['Popularity']])
test.replace(to_replace = np.nan, value =0,inplace=True)

In [11]:
def get_digits(decimal_number):
    #remove sign from number
    decimal_number_absolute = abs(decimal_number)
    #round the number to 2 decimal places
    decimal_number_rounded = round(decimal_number_absolute,2)
    decimal_number_string = str(decimal_number_rounded)
    number_whole=decimal_number_string.split('.')[0]
    number_remainder=decimal_number_string.split('.')[1]
    return len(number_whole) + len(number_remainder)
#test
number = -06.659857487
print(get_digits(number))

3


In [12]:
# Preprocessing and feature extraction
dur = test['duration_in min/ms']
track = []
clean = []
for i in dur:
    if i<=100 :
        clean.append(i)
        track.append(0)
    else:
        clean.append(i/60000)
        track.append(1)
test['duration_in min/ms'] = clean
test['new1'] = track

In [14]:
train.nunique()

Artist Name            9149
Track Name            15129
Popularity              101
danceability            909
energy                 1208
key                      12
loudness               9002
mode                      2
speechiness            1194
acousticness           3930
instrumentalness       4254
liveness               1459
valence                1320
tempo                 13562
duration_in min/ms    13868
time_signature            4
Class                    11
new1                      2
dtype: int64

In [15]:
train.columns

Index(['Artist Name', 'Track Name', 'Popularity', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_in min/ms', 'time_signature', 'Class', 'new1'],
      dtype='object')

In [18]:
cat_cols = ['Artist Name', 'Track Name']
cont_cols = ['Popularity', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_in min/ms', 'time_signature', 'new1']

In [19]:
X_train = train[cat_cols+cont_cols]
y_train = train.Class
X_test = test[cat_cols+cont_cols]

In [25]:
n_folds = 10
subbed = []
kf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=10)


for fold, (train_idx, test_idx) in enumerate(kf.split(X_train, y_train)):
    print('=============== Fold No:',fold+1,'===============')
    X_tr, X_tst = X_train.iloc[train_idx], X_train.iloc[test_idx]
    y_tr, y_tst = y_train.iloc[train_idx], y_train.iloc[test_idx]
    
    model = CatBoostClassifier(n_estimators=10000, random_state=10, eval_metric= 'MultiClass', cat_features=cat_cols)
    model.fit(X_tr, y_tr,eval_set=[(X_tst, y_tst)], early_stopping_rounds=30, verbose=500)
    print(log_loss(y_tst, model.predict_proba(X_tst)))
    subbed.append(log_loss(y_tst, model.predict_proba(X_tst)))
    pred = model.predict_proba(X_test)

=============== Fold No: 1 ===============
Learning rate set to 0.047371
0:	learn: 2.3037980	test: 2.3057627	best: 2.3057627 (0)	total: 264ms	remaining: 43m 58s
500:	learn: 0.7796143	test: 0.7786168	best: 0.7786168 (500)	total: 1m 2s	remaining: 19m 50s
Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.7627750205
bestIteration = 946

Shrink model to first 947 iterations.
0.7627750204855843
=============== Fold No: 2 ===============
Learning rate set to 0.047371
0:	learn: 2.3035826	test: 2.3077990	best: 2.3077990 (0)	total: 119ms	remaining: 19m 53s
500:	learn: 0.7762308	test: 0.7790240	best: 0.7790240 (500)	total: 1m 4s	remaining: 20m 14s
Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.7698111634
bestIteration = 670

Shrink model to first 671 iterations.
0.7698111634218131
=============== Fold No: 3 ===============
Learning rate set to 0.047371
0:	learn: 2.3045938	test: 2.3040426	best: 2.3040426 (0)	total: 186ms	remaining: 31m
500:	learn: 0.7791369	t

In [26]:
out = pd.DataFrame(pred)
out

,0,1,2,3,4,5,6,7,8,9,10
0,0.000588,0.383374,0.010723,0.000213,0.000274,0.000825,0.138285,0.000073,0.010471,0.016986,0.438189
1,0.000070,0.012778,0.005247,0.000008,0.000041,0.002197,0.029505,0.000002,0.697701,0.003830,0.248621
2,0.000055,0.001967,0.000116,0.000008,0.000320,0.008014,0.004770,0.000006,0.004034,0.939357,0.041353
3,0.000236,0.003222,0.000717,0.000044,0.000127,0.021380,0.024853,0.000024,0.003094,0.892445,0.053856
4,0.001472,0.000274,0.000165,0.000074,0.002041,0.000306,0.000507,0.000018,0.001011,0.979992,0.014139
...,...,...,...,...,...,...,...,...,...,...,...
7708,0.000049,0.000469,0.000761,0.000008,0.000027,0.004874,0.002337,0.000005,0.000306,0.985162,0.006002
7709,0.000329,0.082857,0.048134,0.000058,0.000017,0.002635,0.091619,0.000157,0.340850,0.012455,0.420888
7710,0.107099,0.014599,0.005843,0.091240,0.041480,0.039499,0.028329,0.005147,0.001178,0.535236,0.130350
7711,0.000391,0.115703,0.018333,0.000045,0.000140,0.002898,0.477281,0.000042,0.000858,0.034141,0.350168


In [27]:
out = out.rename(columns={0:'Acoustic/Folk_0',
               1:'Alt_Music_1',
               2:'Blues_2',
               3:'Bollywood_3',
               4:'Country_4',
               5:'HipHop_5',
               6:'Indie Alt_6',
               7:'Instrumental_7',
               8:'Metal_8',
               9:'Pop_9',
               10:'Rock_10'})

In [28]:
out.to_csv('out6.csv', index= False)

In [ ]:
def binningFunction(col, cut_poitns, labels=None):
    minval = col.min()
    maxval = col.max()
    break_points = [minval] + cut_points + [maxval]
    print(break_points)
    if not labels:
        labels = range(len(cut_points)+1)
    colBin = pd.cut(col, bins=break_points, labels=labels,include_lowest=True)
    return colBin